In [2]:
import pandas as pd
import numpy as np
from decimal import Decimal
from ans import generate_string#, code_rabs, decode_rabs

np.random.seed(1)

In [398]:
from math import floor

def code_rans(msg, alphabet, freqs, quant_bits=12, renorm_bits=16):
    '''
         msg - list of strings where each character should be in alphabet
         alphabet - list of strings representing symbols in alphabet
         freqs - List of integers f[s] s.t. p_s ~= f[s] / 2^quant_bits
         quant_bits - exponent of 2^N (quantizing factor)
         renorm_bits - n-bit renormalization
    '''
    assert len(freqs) == len(alphabet)
    assert all([type(f) == int for f in freqs])
    assert sum(freqs) == 1 << quant_bits
    assert quant_bits <= renorm_bits
    
    cdf = []
    for i in range(len(freqs) + 1):
        cdf.append(sum(freqs[:i]))
        
    assert len(cdf) == len(freqs) + 1

    codes = []
    code = 1 << renorm_bits
    for x in msg:
        pcode = code
        index = alphabet.index(x)
        
        # Renormalization - if we would push past 2**renorm_bits, then renorm
        new_code = ((floor(code / freqs[index]) << quant_bits)
                    + (code % freqs[index])
                    + cdf[index])
        if new_code > ((1 << (2 * renorm_bits)) - 1):
            codes.append(code & ((1 << renorm_bits) - 1))
            code = code >> renorm_bits
            
        # rANS
        code = ((floor(code / freqs[index]) << quant_bits)
                + (code % freqs[index])
                + cdf[index]) 
        
            
    codes.append(code) 
    return codes

In [377]:
def decode_rans(codes, alphabet, freqs, quant_bits=8, renorm_bits=16):
    '''
         codes - coded message
         alphabet - list of strings representing symbols in alphabet
         freqs - List of integers f[s] s.t. p_s ~= f[s] / 2^quant_bits
         quant_bits - exponent of 2^N (quantizing factor)
         renorm_bits - n-bit renormalization
    '''
    assert len(freqs) == len(alphabet)
    assert all([type(f) == int for f in freqs])
    assert sum(freqs) == (1 << quant_bits)
    assert len(codes) >= 1
    assert all(c < (1 << renorm_bits) for c in codes[:-1])
    assert codes[-1] < (1 << (2*renorm_bits))
    
    codes = codes.copy()
    
    cdf = []
    for i in range(len(freqs) + 1):
        cdf.append(sum(freqs[:i]))
    assert len(cdf) == len(freqs) + 1

    msg = []
    mask = (1 << quant_bits) - 1
    code = codes.pop()
    while code > (1 << renorm_bits):

        pcode = code
        s = code & mask
        index = np.argmax(np.array(cdf) > s) - 1
        pcode = code
        code = (freqs[index] * (code >> quant_bits)
                + (code & mask)
                - cdf[index])
        msg = [alphabet[index]] + msg
        
        if (code < (1 << renorm_bits)) and codes:
            assert codes[-1] < (1 << renorm_bits)
            code = (code << renorm_bits) + codes.pop()
            
       
    assert not codes
    return msg


ALPHABET = ['a', 'b', 'c', 'd']
freqs = [4, 1, 2, 9]
quant_bits = 4
s = generate_string(325, alphabet=ALPHABET, prob=np.array(freqs) / (1 << quant_bits))
#s = ['a', 'd', 'c', 'd', 'a', 'a', 'd', 'd', 'b', 'a', 'a', 'a', 'a', 'd', 'd', 'd', 'd', 'd', 'd', 'c', 'b', 'd', 'c', 'd', 'b', 'd', 'c', 'b', 'd', 'a']
#s = ['a', 'b', 'c']
#s = ['c', 'a']
#s = ['d', 'b', 'c', 'c', 'b', 'a', 'd', 'd', 'b', 'a', 'a', 'b', 'a', 'a', 'd']
#s = ['b', 'd', 'c', 'd', 'd', 'd', 'a', 'a', 'd', 'a', 'd', 'c', 'c', 'a', 'a']
codes = code_rans(s, ALPHABET, freqs=freqs, quant_bits=quant_bits)
decode_msg = decode_rans(codes, ALPHABET, freqs=freqs, quant_bits=quant_bits)

#print(s)
print(codes)
orig = (len(s) * np.log2(len(ALPHABET))) / len(s)
print('Orig bits: ', orig)
actual = (16 * (len(codes) - 1) + np.log2(codes[-1])) / len(s)
print('Actual bits: %.4f' % actual)
print('Ideal bits: %.4f' % sum([-f / (1 << quant_bits) * np.log2(f / (1 << quant_bits)) for f in freqs]))
print('Compression ratio: %.4f' % (orig / actual))
print(16 * (len(codes) - 1) + np.log2(codes[-1]))
#print(decode_msg)
assert s == decode_msg

[8729, 16261, 27134, 61231, 38415, 21410, 42929, 22032, 27861, 3985, 42371, 65072, 47707, 62830, 13851, 51761, 24795, 52513, 34869, 26945, 40986, 1321, 5244, 4154, 65443, 886, 41987, 24555, 3348, 13814, 8788, 3964903429]
Orig bits:  2.0
Actual bits: 1.6243
Ideal bits: 1.5919
Compression ratio: 1.2313
527.8846385814037


In [323]:
#1 << 16
#3393540112 / 1508299780
#1508299780 > (1 << 32)
#6033199120 > (1 << 32)
#((92059 << 16) + 20496) / 2
#1508299780 & ((1 << 16) - 1)
#6033199120 & ((1 << 16) - 1)
6033199120 >> 16 #& ((1 << 16) - 1)

92059

In [162]:
#def benchmark_uabs(p, msg_len, N, ALPHABET=('a', 'b')):
#    print("p: ", p)
#    print("Message Length: ", msg_len)
#    print("No. of trials: ", N)
#    entropy = []
#    for i in range(N):
#        msg = generate_binary_string(msg_len, alphabet=ALPHABET, prob=p)
#        code = code_uabs(msg, alphabet=ALPHABET, prob=p)
#        entropy.append(np.log2(float(code)))
#        
#        decoded_msg = decode_uabs(code, alphabet=ALPHABET, prob=p)
#        
#        if decoded_msg != msg:
#            print("Message: ", msg)
#            print("Code: ", code)
#            print("Entropy: %.2f bits (Orig: %.2f)" % (np.log2(float(code)), len(msg)))
#            print("Decoded: ", decoded_msg)
#            print("Matching: ", decoded_msg == msg)
#            break
#            
#    print("Actual Entropy Stats: ")
#    display(pd.DataFrame(entropy).describe())
#    print("Actual Bits: %.2f" % pd.Series(entropy).mean())
#    
#    prob = [float(x) for x in p]
#    print("Ideal Bits: %.2f" % (-np.array(prob) * np.log2(np.array(prob)) * msg_len).sum())
#    print("Actual Compression Rate: %.2f" % (msg_len / pd.Series(entropy).mean()))
#    print("Ideal Compression Rate: %.2f" % (msg_len / (-np.array(prob) * np.log2(np.array(prob)) * msg_len).sum()))

In [378]:
-4/7*np.log2(4/7) -2/7*np.log2(2/7) -1/7*np.log2(1/7)

1.3787834934861753

In [380]:
round(-np.log2(1/3), 4)

1.585

In [383]:
np.log2(51)

5.672425341971495

In [412]:
#ALPHABET = ['a', 'b', 'c']
#freqs = [3, 3, 2]
#quant_bits = 3
#s = ['a']
#code_rans(s, ALPHABET, freqs=freqs, quant_bits=quant_bits, renorm_bits=8)[0]

425

In [601]:
freqs = [5, 2, 1]
CDF = [sum(freqs[:0]), sum(freqs[:1]), sum(freqs[:2]), sum(freqs)]
n = 3

# C(x, s)
print(freqs)
print(x)
s=2
for x in range(40, 50):
    #print(floor(x / freqs[s]), (floor(x / freqs[s]) << n), (x % freqs[s]), CDF[s])
    prex = x
    x = (floor(x / freqs[s]) << n) + (x % freqs[s]) + CDF[s]
    print(prex, " ===> ", x)

[5, 2, 1]
159
40  ===>  327
41  ===>  335
42  ===>  343
43  ===>  351
44  ===>  359
45  ===>  367
46  ===>  375
47  ===>  383
48  ===>  391
49  ===>  399
